# Aggregations
Su objetivo es resumir y agrupar información númerica de grandes conjuntos de información, para ello es necesario definir: una llave o grouping y una función de Agregación.
Existen muchas funciones de agregación en Apache Spark, que son de vital importancia cuando se requiere hacer analisis de los datos. 
Entre los grupos tenemos Simples, Grouping,  Window, GroupingSet, Rollup y Cube.
En este notebook vamos a ir describiendo cada uno de los grupos con ejemplos que podran ejecutar.

## Simple

Son las funciones de Agregación mas sencillas, entregan información resumida de un conjunto de información completo. 
Todas las funciones se pueden encontrar aca: https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$
A continuación se muestran algunas de las más populares.

### count
En este caso vamos a ver que este count no va a ser una action si no una transformación, podemos definir si queremos hacer como en SQL  un count(*), count(1) o un count("columna"). 

Vamos a tomar un conjunto de datos https://www.kaggle.com/sidtwr/videogames-sales-dataset para trabajar con estas funciones.

In [ ]:
//Import para Jupyter-notebooks 
import $ivy.`org.apache.spark::spark-sql:2.4.0`
import $ivy.`sh.almond::almond-spark:0.6.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

In [ ]:
val df = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("resources/vgsales.csv")


In [ ]:
df.createOrReplaceTempView("vgsales")

In [ ]:
import org.apache.spark.sql.functions.count
df.select(count("*")).show()

In [ ]:
spark.sql("select count(*) from vgsales").show()

### CountDistinct

En ocaciones queremos contar unicamente los registros que son únicos en una columna especifica.

In [ ]:
import org.apache.spark.sql.functions.countDistinct
time(df.select(countDistinct("Genre")).show())

In [ ]:
spark.sql("select count(distinct Genre) from vgsales").show()

### approx_count_distinct
Cuando se trabaja con grandes volumenes de información, tener un número que se aproxime a la cantidad real es valioso, con esta función podemos definir el máximo error permitido. 

In [ ]:
import org.apache.spark.sql.functions.approx_count_distinct
time(df.select(approx_count_distinct("Genre", 0.1).as("approxCount")).show() )

In [ ]:
spark.sql("select APPROX_COUNT_DISTINCT(Genre) as approxCount from vgsales").show()

El tiempo que se tomo el count exacto fue un poco mas de 3segundos y este ulitmo solo 0.2 segundos

### first and last
Esta función permite retornar el primer y ultimo registro de nuestro dataset.

In [ ]:
import org.apache.spark.sql.functions.{first, last}
df.select(first("Name"), last("Name")).show()

In [ ]:
spark.sql("select first(name), last(name) from vgsales").show()

### min and max
Obtiene el valor mínimo y máximo de la columna deseada. 

In [ ]:
import org.apache.spark.sql.functions.{min, max,col}
df.where(col("Publisher") === "Nintendo").select(min("Global_Sales"), max("Global_Sales")).show()

In [ ]:
spark.sql("""
SELECT min(Global_Sales), max(Global_Sales) 
FROM vgsales WHERE Publisher = 'Nintendo'
""").show()

### sum
Realiza la sumatoria de todos los valores en una sola fila

In [ ]:
import org.apache.spark.sql.functions.sum
df.select(sum("Global_Sales")).show()

In [ ]:
spark.sql("select sum(Global_Sales) from vgsales").show()

### sumDistinct
Adicional a la suma de todos los registros, puedes sumar únicamente los valores distintos.

In [ ]:
import org.apache.spark.sql.functions.sumDistinct
df.select(sumDistinct("Global_Sales")).show() 

### Average (avg o mean)
En otro escenario tendriamos que primero sumar todos lo registros y dividirlos por el *count*, Spark provee una manera mas facirl de obtener el promedio. 

In [ ]:
import org.apache.spark.sql.functions.{sum, count, avg, expr}

df.select(
    count("*").alias("# video games"),
    sum("NA_Sales").alias("North America Sales"),
    avg("NA_Sales").alias("avg NA Sales"),
    expr("mean(NA_Sales)").alias("mean NA Sales")).show()

In [ ]:
spark.sql("""
SELECT count(*) as num_video_games, sum(NA_Sales) as NorthAmericaSales, avg(NA_Sales) 
FROM vgsales
""").show()

### Varianza y desviación estándar 
Spark ofrece estas y algunas otras funciones mas especificas para analisis estadisticos*. Para estas se pueden calcular muestrales o poblacionales.
* Asimetria y curtosis  `org.apache.spark.sql.functions.{skewness, kurtosis}`
* Covarianza y correlación `import org.apache.spark.sql.functions.{corr, covar_pop, covar_samp}`

In [ ]:
import org.apache.spark.sql.functions.{var_pop, stddev_pop}
import org.apache.spark.sql.functions.{var_samp, stddev_samp}
df.select(var_pop("Global_Sales"), var_samp("Global_Sales"),
  stddev_pop("Global_Sales"), stddev_samp("Global_Sales")).show()

Por último, en Spark se puede obtener información general de cada columna con describe

In [ ]:
df.select(col("Platform"), col("Global_Sales")).describe().show()